# Advanced InCA Features

This notebook demonstrates the advanced features of the InCA system:
1. Active Learning
2. Drift Detection
3. Performance Optimization

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
sys.path.append('../../')

from src.inca_agent import InCAAgent
from src.llm_integration.gpt4_handler import GPT4Handler
from src.utils.active_learning import ActiveLearner
from src.utils.drift_detector import DriftDetector
from src.utils.performance_optimizer import PerformanceOptimizer

## 1. Active Learning

Demonstrate how active learning selects informative samples.

In [ ]:
# Initialize components
active_learner = ActiveLearner()
llm_handlers = {'gpt4': GPT4Handler(os.getenv('OPENAI_API_KEY'))}
agent = InCAAgent(llm_handlers)

# Generate some candidate samples
candidates = [
    {
        'text': 'Complex technical issue with database indexing',
        'confidences': {'technical': 0.6, 'business': 0.4},
        'embedding': llm_handlers['gpt4'].get_embeddings('Complex technical issue')
    },
    {
        'text': 'Market analysis shows growth potential',
        'confidences': {'technical': 0.3, 'business': 0.7},
        'embedding': llm_handlers['gpt4'].get_embeddings('Market analysis')
    },
    {
        'text': 'System performance impacts revenue',
        'confidences': {'technical': 0.5, 'business': 0.5},
        'embedding': llm_handlers['gpt4'].get_embeddings('System performance')
    }
]

# Select informative samples
selected_samples = active_learner.select_samples(candidates, n_samples=2)

print("Selected samples for labeling:")
for sample in selected_samples:
    print(f"\nText: {sample.text}")
    print(f"Uncertainty: {sample.uncertainty:.2f}")
    print(f"Diversity Score: {sample.diversity_score:.2f}")
    print(f"Combined Score: {sample.combined_score:.2f}")

## 2. Drift Detection

Monitor and detect concept drift in the system.

In [ ]:
# Initialize drift detector
drift_detector = DriftDetector()

# Add reference data
reference_embeddings = [
    llm_handlers['gpt4'].get_embeddings(text)
    for text in [
        'Technical documentation review',
        'System architecture analysis',
        'Database optimization required'
    ]
]

drift_detector.update_reference_window('technical', reference_embeddings)

# Test for drift with new data
current_embeddings = [
    llm_handlers['gpt4'].get_embeddings(text)
    for text in [
        'Code review feedback',
        'Performance optimization needed',
        'Technical debt assessment'
    ]
]

# Detect drift
drift_metrics = drift_detector.detect_drift('technical', current_embeddings)

if drift_metrics:
    print(f"Drift detected!")
    print(f"Drift score: {drift_metrics.drift_score:.2f}")
    print(f"Severity: {drift_metrics.severity}")
    print("\nRecommendations:")
    for rec in drift_detector.get_drift_recommendations(drift_metrics):
        print(f"- {rec}")
else:
    print("No significant drift detected")

## 3. Performance Optimization

Demonstrate performance optimization techniques.

In [ ]:
# Initialize optimizer
optimizer = PerformanceOptimizer()

# Test batch processing
test_texts = [
    "Technical review needed",
    "Performance optimization required",
    "System architecture analysis",
    "Database indexing improvement"
]

def process_text(text: str) -> List[float]:
    return llm_handlers['gpt4'].get_embeddings(text)

# Process in batches
results = optimizer.batch_process(test_texts, process_text)

# Get optimization metrics
metrics = optimizer.get_optimization_metrics()
print("\nOptimization Metrics:")
print(f"Cache Hit Rate: {metrics.cache_hit_rate:.2f}")
print(f"Average Response Time: {metrics.avg_response_time:.2f}s")
print(f"Memory Usage: {metrics.memory_usage_mb:.2f}MB")
print(f"Throughput: {metrics.throughput:.2f} requests/second")

# Get optimization recommendations
print("\nOptimization Recommendations:")
for rec in optimizer.get_optimization_recommendations():
    print(f"- {rec}")